In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float

# Load the data from the provided CSV files
drivers_df = pd.read_csv('Sources/drivers_updated.csv')
teams_df = pd.read_csv('Sources/teams_updated.csv')
winners_df = pd.read_csv('Sources/winners.csv')

# Process the data before loading into the database
# Extract the year from the Date column in winners_df
winners_df['Year'] = pd.to_datetime(winners_df['Date']).dt.year

# Split the 'Name Code' column to extract the 'Code'
winners_df['Code'] = winners_df['Name Code'].str.extract(r'(\b[A-Z]{3}\b)')

# Rename the 'Car' column to 'Team' in winners_df
winners_df.rename(columns={'Car': 'Team'}, inplace=True)

# Rename the 'Team' column to 'Car' in teams_df
teams_df.rename(columns={'Team': 'Car'}, inplace=True)

# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()

# Define tables based on the loaded dataframes
drivers = Table('drivers', metadata,
                Column('Pos', Integer),
                Column('Driver', String),
                Column('Nationality', String),
                Column('Car', String),
                Column('PTS', Float),
                Column('year', Integer),
                Column('Code', String))

teams = Table('teams', metadata,
              Column('Pos', Integer),
              Column('Car', String),
              Column('PTS', Float),
              Column('year', Integer))

winners = Table('winners', metadata,
                Column('Grand Prix', String),
                Column('Date', String),
                Column('Winner', String),
                Column('Team', String),
                Column('Laps', Float),
                Column('Time', String),
                Column('Name Code', String),
                Column('Year', Integer),
                Column('Code', String))

# Create tables in the database
metadata.create_all(engine)

# Load data into the database
drivers_df.to_sql('drivers', engine, if_exists='append', index=False)
teams_df.to_sql('teams', engine, if_exists='append', index=False)
winners_df.to_sql('winners', engine, if_exists='append', index=False)

# Calculate the total number of victories for each driver
driver_victories = winners_df['Winner'].value_counts().reset_index()
driver_victories.columns = ['Driver', 'Total_Victories_Driver']

# Calculate the total number of victories for each team
team_victories = winners_df['Team'].value_counts().reset_index()
team_victories.columns = ['Team', 'Total_Victories_Team']

# Merge the dataframes
merged_df = pd.merge(drivers_df, driver_victories, how='left', left_on='Driver', right_on='Driver')
merged_df = pd.merge(merged_df, team_victories, how='left', left_on='Car', right_on='Team')

# Add the total driver victories column
merged_df['Total_Victories_Driver'] = merged_df['Total_Victories_Driver'].fillna(0)

# Add the total team victories column
merged_df['Total_Victories_Team'] = merged_df['Total_Victories_Team'].fillna(0)

# Fill missing data with 'NA'
merged_df.fillna('NA', inplace=True)

# Drop duplicate or unnecessary columns
merged_df.drop(columns=['Pos', 'PTS', 'year', 'Code'], inplace=True)

# Rename columns to remove suffixes and clarify their names
merged_df.rename(columns={'Car': 'Team'}, inplace=True)

# Remove duplicates to ensure each driver appears only once
merged_df.drop_duplicates(subset=['Driver'], keep='first', inplace=True)

# Remove all rows that have zero in Total_Victories_Driver
merged_df = merged_df[merged_df['Total_Victories_Driver'] != 0]

# Save the merged dataframe to a new CSV file
merged_df.to_csv('Sources/merged_results.csv', index=False)

# Display the merged dataframe
merged_df.head()



,Driver,Nationality,Team,Total_Victories_Driver,Team,Total_Victories_Team
0,Nino Farina,ITA,Alfa Romeo,5.0,Alfa Romeo,11.0
1,Juan Manuel Fangio,ARG,Alfa Romeo,24.0,Alfa Romeo,11.0
2,Luigi Fagioli,ITA,Alfa Romeo,1.0,Alfa Romeo,11.0
3,Louis Rosier,FRA,Talbot-Lago,0.0,NA,0.0
4,Alberto Ascari,ITA,Ferrari,13.0,Ferrari,245.0
